In [1]:
import pandas as pd
pd.set_option("display.max_rows", 5)
reviews = pd.read_csv("./input/winemag-data-130k-v2.csv", index_col=0)

reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Exercises

## 1.

What is the median of the `points` column in the `reviews` DataFrame?

In [2]:
median_points = reviews.points.median()

median_points

np.float64(88.0)

## 2. 
What countries are represented in the dataset? (Your answer should not include any duplicates.)

数据集中包含哪些国家？（答案不得包含重复内容）。

In [3]:
countries = reviews.country.unique()

countries

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru',
       'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
       'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia',
       'China', 'Egypt'], dtype=object)

## 3.
How often does each country appear in the dataset? Create a Series `reviews_per_country` mapping countries to the count of reviews of wines from that country.

每个国家在数据集中出现的频率是多少？创建`reviews_per_country`系列，将country映射到该国葡萄酒的reviews数。

In [4]:
reviews_per_country = reviews.country.value_counts()
reviews_per_country

country
US        54504
France    22093
          ...  
China         1
Egypt         1
Name: count, Length: 43, dtype: int64

In [5]:
type(reviews_per_country)

pandas.core.series.Series

## 4.
Create variable `centered_price` containing a version of the `price` column with the mean price subtracted.

创建变量 `centered_price` 包含减去平均价格后的 `price` 列版本。

(Note: this 'centering' transformation is a common preprocessing step before applying various machine learning algorithms.) 

（注：这种'居中'转换是应用各种机器学习算法前的常见预处理步骤）。

In [6]:
centered_price = reviews.price - reviews.price.mean()
centered_price

0               NaN
1        -20.363389
            ...    
129969    -3.363389
129970   -14.363389
Name: price, Length: 129971, dtype: float64

## 5.
I'm an economical wine buyer. Which wine is the "best bargain"? Create a variable `bargain_wine` with the title of the wine with the highest points-to-price ratio in the dataset.

我是一个经济型葡萄酒买家。哪种酒 "最便宜"？创建一个变量`bargain_wine`，其中包含数据集中points-to-price比例最高的葡萄酒的名称。

In [7]:
bargain_idx = (reviews.points / reviews.price).idxmax()
bargain_idx

np.int64(64590)

In [8]:
bargain_wine = reviews.loc[bargain_idx, "title"]
bargain_wine

'Bandit NV Merlot (California)'

## 6.
There are only so many words you can use when describing a bottle of wine. Is a wine more likely to be "tropical" or "fruity"? Create a Series `descriptor_counts` counting how many times each of these two words appears in the `description` column in the dataset. (For simplicity, let's ignore the capitalized versions of these words.)

在描述一瓶葡萄酒时，你能用到的词汇有限。一款葡萄酒更可能是 "热带风味"还是 "果味"？创建一个`descriptor_counts`系列，计算这两个词分别在数据集的`description`列中出现的次数。（简单起见，忽略这些词的大写版本）。

In [9]:
tropical_counts = reviews.description.map(lambda d: "tropical" in d).sum()
fruity_counts = reviews.description.map(lambda d: "fruity" in d).sum()

print(f"tropical_counts = {tropical_counts}, fruity_counts = {fruity_counts}")

tropical_counts = 3607, fruity_counts = 9090


In [10]:
descriptor_counts = pd.Series([tropical_counts, fruity_counts], index=['tropical', 'fruity'])
descriptor_counts

tropical    3607
fruity      9090
dtype: int64

## 7.
We'd like to host these wine reviews on our website, but a rating system ranging from 80 to 100 points is too hard to understand - we'd like to translate them into simple star ratings. A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.

我们想把这些酒评放在我们的网站上，但从 80 分到 100 分的评分系统太难懂了--我们想把它们转换成简单的星级评分。95 分或以上为 3 星，85 分以上但低于 95 分为 2 星。其他分数为 1 星。

Also, the Canadian Vintners Association bought a lot of ads on the site, so any wines from Canada should automatically get 3 stars, regardless of points.

此外，加拿大葡萄酒商协会在该网站上购买了大量广告，因此任何来自加拿大的葡萄酒都会自动获得 3 星，无论分数高低。

Create a series `star_ratings` with the number of stars corresponding to each review in the dataset.

创建`star_ratings`系列，其中包含数据集中每篇评论对应的星级数。

In [11]:
def stars(row):
    if row.country == 'Canada':
        return 3
    elif row.points >= 95:
        return 3
    elif row.points >= 85:
        return 2
    else:
        return 1
    
star_ratings = reviews.apply(stars, axis='columns')
star_ratings

0         2
1         2
         ..
129969    2
129970    2
Length: 129971, dtype: int64